In [1443]:
import sqlite3 
from sqlite3 import Error
import pandas as pd

> Дано 2 CSV-файла: один с транзакциями за период 01.01.2023 – 20.04.2023, второй с информацией о клиентах.
> 
> **За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**
> 
> Необходимо в Jupyter- ноутбуке выполнить следующие пункты, используя SQLite:

## Шаг 1

> **(Балл - 2)** Необходимо скачать CSV-файл - <span style="color:green">transactions_for_dz2.csv</span>, создать таблицу _transaction_bd_ со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:
> 
> <br/>
> (0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции<br/>
> (1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции<br/>
> (2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента<br/>
> (3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала<br/>
> (4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции<br/>
> 
> <br/>
> Также необходимо скачать второй CSV-файл - <span style="color:green">client_info.csv</span>, создать таблицу _customer_bd_ со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:
> <br/>
> <br/>
> (0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента<br/>
> (1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте<br/>
> (2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте<br/>
> (3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента<br/>
> (4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента<br/>
> <br/>
> 
> **Примечание для следующих шагов**: сумма транзакций не может быть null-ом, это всегда число.

Создаем функцию для создания подключения с возможностью создания БД в памяти и в файле. По-умолчанию будем использовать создание БД в памяти

In [1444]:
def create_connection(in_memory=True, path=None):
    conn = None
    try:
        conn = sqlite3.connect(":memory:" if in_memory else path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return conn

Создаем функцию для выполнения запросов

In [1445]:
def execute_query(query, conn):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

Создаем функцию для заполнения таблицы БД данными из датасета

In [1446]:
def sql_insert(data, name, conn, m=0, n=1):
    cursor = conn.cursor()
    try:
        cursor.executemany(f'''
                INSERT INTO {name} {tuple(data.columns[m:n])} 
                    VALUES ({str("?, " * (n-m-1) + "?")})
                    ''',
                data.iloc[:, m:n].values)
        conn.commit()
        print('Data inserted successfully')
    except Error as e:
        print(f"The error '{e}' occurred")

Создаем функцию для получения информации по столбцам

In [1447]:
def table_info(table_name, conn):
    c = conn.cursor()
    meta = c.execute("PRAGMA table_info('" + table_name + "')")
    for r in meta:
        print(r)

Задаем дату по-умолчанию

In [1448]:
default_date = '2023-05-01'

Задаем имя таблицы БД с транзакциями

In [1449]:
transaction_table = 'transaction_bd'

Задаем имя таблицы БД с клиентами

In [1450]:
customer_table = 'customer_bd'

Создаем подключение

In [1451]:
conn = create_connection()

Connection to SQLite DB successful


Создаем таблицу транзакций в БД.<br/>
По условию, колонки должны иметь разные типы. Посмотрим на полученный результат и типы столбцов после загрузки данных из датасета.

In [1452]:
execute_query(
    f'''
    CREATE TABLE IF NOT EXISTS {transaction_table} (
        transaction_id INTEGER,
        tx_datetime NUMERIC,
        customer_id INTEGER,
        terminal_id INTEGER,
        tx_amount REAL
    );
    ''',
    conn)

Query executed successfully


Загружаем данные из csv в датасет

In [1453]:
trans_df = pd.read_csv('data/transactions_for_dz2.csv', delimiter=',')

Посмотрим на датасет

In [1454]:
trans_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [1455]:
trans_df.tail()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44
1048574,1048574,2023-04-20 10:07:43,3252,6009,304.55


In [1456]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


Загружаем данные из датасета в таблицу БД

In [1457]:
sql_insert(trans_df, transaction_table, conn, 0, trans_df.shape[1])

Data inserted successfully


Проверяем соответствие атрибутов условию

In [1458]:
table_info(transaction_table, conn)

(0, 'transaction_id', 'INTEGER', 0, None, 0)
(1, 'tx_datetime', 'NUMERIC', 0, None, 0)
(2, 'customer_id', 'INTEGER', 0, None, 0)
(3, 'terminal_id', 'INTEGER', 0, None, 0)
(4, 'tx_amount', 'REAL', 0, None, 0)


Выберем 10 записей из таблицы и посмотрим на результат

In [1459]:
pd.read_sql(
    f'''
    SELECT * 
    FROM {transaction_table} 
    LIMIT 10
    ''', 
    conn)

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
5,5,2023-01-01 00:10:45,568,8803,401.17
6,6,2023-01-01 00:11:30,2803,5490,938.54
7,7,2023-01-01 00:11:44,4684,2486,206.53
8,8,2023-01-01 00:11:53,4128,8354,253.47
9,9,2023-01-01 00:13:44,541,6212,555.63


Создаем таблицу клиентов в БД.<br/>
По условию, колонки должны иметь разные типы. Посмотрим на полученный результат и типы столбцов после загрузки данных из датасета.

In [1460]:
execute_query(
    f'''
    CREATE TABLE IF NOT EXISTS {customer_table} (
        client_id INTEGER,
        start_dt NUMERIC,
        end_dt NUMERIC,
        client_name TEXT,
        year_birth TEXT
    );
    ''',
    conn)

Query executed successfully


Загружаем данные из csv в датасет

In [1461]:
client_df = pd.read_csv('data/client_info.csv', delimiter=';')

Посмотрим на датасет

In [1462]:
client_df.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [1463]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


In [1464]:
client_df.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [1465]:
client_df.tail()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946
4987,2023-04-20,2999-12-31,Idalis,2000,2853


In [1466]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


Загружаем данные из датасета в таблицу БД

In [1467]:
sql_insert(client_df, customer_table, conn, 0, client_df.shape[1])

Data inserted successfully


Проверяем соответствие атрибутов условию

In [1468]:
table_info(customer_table, conn)

(0, 'client_id', 'INTEGER', 0, None, 0)
(1, 'start_dt', 'NUMERIC', 0, None, 0)
(2, 'end_dt', 'NUMERIC', 0, None, 0)
(3, 'client_name', 'TEXT', 0, None, 0)
(4, 'year_birth', 'TEXT', 0, None, 0)


Выберем 10 записей из таблицы и посмотрим на результат

In [1469]:
pd.read_sql(
    f'''
    SELECT * 
    FROM {customer_table} 
    LIMIT 10
    ''', 
    conn)

,client_id,start_dt,end_dt,client_name,year_birth
0,2213,2015-07-16,2021-01-01,Olivia,1979
1,1148,2015-07-17,2021-01-01,Emma,1979
2,2293,2015-07-18,2021-01-01,Charlotte,1979
3,1867,2015-07-19,2021-01-01,Amelia,1979
4,1767,2015-07-20,2021-01-01,Ava,1979
5,3853,2015-07-21,2021-01-01,Sophia,1979
6,1605,2015-07-22,2021-01-01,Isabella,1979
7,3273,2015-07-23,2021-01-01,Mia,1979
8,147,2015-07-24,2021-01-01,Evelyn,1979
9,1420,2015-07-25,2021-01-01,Harper,1979


## Шаг 2

> **(Балл - 1 за каждый пункт)** Написать скрипты:

> **a**. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [1470]:
pd.read_sql(
    f'''
    SELECT 
        tmp.customer_id,
        ct.client_name AS client_name, 
        MAX(tmp.tx_sum) AS max_sum, 
        DATE(ct.end_dt) AS end_dt
    FROM (
        SELECT
            customer_id, 
            SUM(coalesce(tx_amount,0)) AS tx_sum
        FROM 
            {transaction_table}
        GROUP BY
            customer_id
        ) AS tmp
    JOIN {customer_table} AS ct
        ON tmp.customer_id = ct.client_id
        AND DATE(ct.end_dt) >= DATE('{default_date}')
    ''', 
conn)

,customer_id,client_name,max_sum,end_dt
0,2891,Kylee,786115.87,2999-12-31


> **b**. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [1471]:
pd.read_sql(
    f'''
    SELECT 
        ct.client_name, 
        MIN(tx.tx_sum) AS min_sum, 
        DATE(ct.end_dt) AS end_dt 
    FROM (
        SELECT
            customer_id, 
            SUM(coalesce(tx_amount,0)) AS tx_sum
        FROM 
            {transaction_table}
        GROUP BY
            customer_id
    ) AS tx 
    LEFT JOIN {customer_table} AS ct
        ON tx.customer_id = ct.client_id
    WHERE DATE(ct.end_dt) >= DATE('{default_date}')
    ''', 
conn)

,client_name,min_sum,end_dt
0,Kloe,30.48,2999-12-31


> **c**. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.

In [1472]:
pd.read_sql(
    f'''
    SELECT
        tx.customer_id, 
        SUM(coalesce(tx.tx_amount,0)) AS tx_sum
    FROM 
        {transaction_table} as tx
    INNER JOIN (
        SELECT
            client_id
        FROM {customer_table}
        WHERE client_id LIKE '111%'
            AND DATE(end_dt) >= DATE('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    WHERE DATE(tx_datetime) < '2023-04-01'
    GROUP BY
        tx.customer_id
    ''', 
conn)

,customer_id,tx_sum
0,1112,35273.16
1,1113,229558.85
2,1114,108622.52
3,1115,38690.76
4,1116,191203.59
5,1117,52751.58
6,1118,353003.39
7,1119,221192.71


> **d**. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.

In [1473]:
pd.read_sql(
    f'''
    SELECT
        ct.year_birth, 
        SUM(coalesce(tx.tx_amount,0)) AS tx_sum
    FROM 
        {transaction_table} AS tx
    INNER JOIN (
        SELECT
            client_id, 
            year_birth
        FROM {customer_table}
        WHERE DATE(end_dt) >= DATE('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    GROUP BY
        ct.year_birth
    ORDER BY
        ct.year_birth DESC
    ''', 
conn)

,year_birth,tx_sum
0,2000,8.632720e+07
1,1999,1.041507e+06
2,1998,6.016750e+05
3,1997,8.479943e+06
4,1996,1.483253e+06
5,1995,6.321200e+06
6,1994,6.244281e+06
7,1993,5.732058e+06
8,1992,5.251433e+06
9,1991,7.141074e+06


> **e**. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [1474]:
pd.read_sql(
    f'''
    SELECT
        ct.client_id, 
        ct.client_name, 
        COUNT(tx.transaction_id) AS tx_count
    FROM 
        {transaction_table} AS tx
    INNER JOIN (
        SELECT
            client_id, 
            client_name
        FROM {customer_table}
        WHERE DATE(end_dt) >= DATE('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    GROUP BY
        ct.client_id
    ORDER BY
        COUNT(tx.transaction_id) DESC
    ''', 
conn)

,client_id,client_name,tx_count
0,2891,Kylee,477
1,2580,Cerenity,464
2,1953,Adara,455
3,2932,Michaela,454
4,3864,Claire,453
...,...,...,...
3731,1976,Becky,1
3732,1942,Perel,1
3733,1880,Roman,1
3734,1095,Marisa,1


## Шаг 3

> **(Балл - 3)** Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [1475]:
pd.read_sql(
    f'''
    SELECT
        strftime('%m', DATE(tx_datetime)) AS tx_month, 
        SUM(coalesce(tx.tx_amount,0)) AS tx_sum
    FROM 
        {transaction_table} AS tx
    INNER JOIN (
        SELECT
            client_id
        FROM {customer_table}
        WHERE DATE(end_dt) >= DATE('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    GROUP BY
        strftime('%m', DATE(tx_datetime))
    ''', 
conn)

,tx_month,tx_sum
0,01,1.159199e+08
1,02,1.062295e+08
2,03,1.150534e+08
3,04,7.261812e+07


Закрываем подключение

In [1476]:
conn.close()